Importar módulos.

In [ ]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

Preâmbulo.

In [ ]:
dataset_path = Path('/home/cefrohlich/github/churn/data/external/Customer-Churn - Customer-Churn.csv')

Importar dados.

In [ ]:
clients = pd.read_csv(
    filepath_or_buffer=dataset_path,
    sep=',',
    header=0,
    decimal=','
)

Analisar duração do contrato para (i) total de clientes e (ii) clientes que encerraram seus contratos.

In [ ]:
churners = clients.loc[clients['Churn'] == 'Yes']

fig, axs = plt.subplots(
    nrows=1,
    ncols=2,
    sharex=True,
    sharey=True
)

fig.set_size_inches(18, 9)

n_bins = 12

axs[0].hist(
    clients['tenure'],
    bins=n_bins
)

axs[0].title.set_text('Total de clientes')

axs[1].hist(
    churners['tenure'],
    bins=n_bins
)

axs[1].title.set_text('Churners')

plt.show()

Estes dados parecem sugerir que os contratos tendem a ser curtos, i.e., que o churning tende a ser maior nos meses iniciais do contrato. Calculemos o risco de churning ao longo da duração do contrato.

In [ ]:
clients_by_tenure = clients[['customerID', 'tenure']].groupby(by='tenure').count()
clients_by_tenure.rename(
    mapper={'customerID': 'clients'},
    axis=1,
    inplace=True
)
clients_by_tenure.reset_index(drop=False, inplace=True)

churners_by_tenure = churners[['customerID', 'tenure']].groupby(by='tenure').count()
churners_by_tenure.rename(
    mapper={'customerID': 'churners'},
    axis=1,
    inplace=True
)
churners_by_tenure.reset_index(drop=False, inplace=True)

churning_rate_by_tenure = clients_by_tenure.merge(
    right=churners_by_tenure,
    how='outer',
    on='tenure'
)

churning_rate_by_tenure['churning_rate'] = churning_rate_by_tenure['churners'] / churning_rate_by_tenure['clients']

fig, ax = plt.subplots()

fig.set_size_inches(18, 9)

ax.plot(
    churning_rate_by_tenure['tenure'],
    churning_rate_by_tenure['churning_rate']
)

plt.show()

De fato, o risco de churning é superior a 50% no início do contrato.

Em contrapartida, o risco de churning gravita em torno de 10% quando se trata de clientes com 60 meses de contrato.

Será o risco de churning diferente para clientes de fibra óptica? Examinemo-lo.

In [ ]:
clients_fiber_optic = clients.loc[clients['InternetService'] == 'Fiber optic']
clients_fiber_optic_by_tenure = clients_fiber_optic[['customerID', 'tenure']].groupby(by='tenure').count()
clients_fiber_optic_by_tenure.rename(
    mapper={'customerID': 'clients'},
    axis=1,
    inplace=True
)
clients_fiber_optic_by_tenure.reset_index(drop=False, inplace=True)

churners_fiber_optic = churners.loc[churners['InternetService'] == 'Fiber optic']
churners_fiber_optic_by_tenure = churners[['customerID', 'tenure']].groupby(by='tenure').count()
churners_fiber_optic_by_tenure.rename(
    mapper={'customerID': 'churners'},
    axis=1,
    inplace=True
)
churners_fiber_optic_by_tenure.reset_index(drop=False, inplace=True)

churning_rate_fiber_optic_by_tenure = clients_fiber_optic_by_tenure.merge(
    right=churners_fiber_optic_by_tenure,
    how='outer',
    on='tenure'
)

churning_rate_fiber_optic_by_tenure['churning_rate'] = churning_rate_fiber_optic_by_tenure['churners'] / churning_rate_fiber_optic_by_tenure['clients']

fig, ax = plt.subplots()

fig.set_size_inches(18, 9)

ax.plot(
    churning_rate_by_tenure['tenure'],
    churning_rate_by_tenure['churning_rate'],
    label='Total de clientes'
)

ax.plot(
    churning_rate_fiber_optic_by_tenure['tenure'],
    churning_rate_fiber_optic_by_tenure['churning_rate'],
    label='Clientes do serviço de fibra óptica'
)

plt.legend()

plt.show()

De fato, clientes do serviço de fibra óptica parecem ser duas vezes mais propensos a encerrar seus contratos do que a média dos clientes. Ademais, o encerramento do contrato é muito mais provável em seu início. Isto indica um problema com a qualidade do serviço de fibra óptica.